In [44]:
!apt-get install gcc
!apt-get install make
!apt-get install libgmp3-dev
!pip install pysmt
!pysmt-install --check
!pysmt-install --msat

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gcc is already the newest version (4:7.4.0-1ubuntu2.3).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgmp3-dev is already the newest version (2:6.1.2+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Installed Solvers:
  msat      True (5.6.1)             
  cvc4      False (None)              Not in Python's path!
  z3        False (None)              Not in Python's path!
  yices     False (None)              Not in Python's path!
  btor      False (None)              Not in Python's path!
  picosat   False (None)              Not in Python's 

In [51]:
from pysmt.shortcuts import Symbol, LE, GE, Int, And, Equals, Plus, Solver, serialize, Not
from pysmt.typing import INT, BOOL
from pysmt.parsing import parse

# Replacement of integer expressions in equation by boolean expressions:
# Before:
#   equation = (1 <= h) & (10 >= h)
# After:
#   equation = A & B
equation = "(1 <= h) & (10 >= h)".lower()
eq_variables = set(v for v in equation if v.isalpha())
int_variables = set(Symbol(s, INT) for s in eq_variables)

# Equation
# formula = (1 <= h) & (10 >= h)
formula = parse(equation)

# Replace atoms by symbolic variables.
# E.g.: 
#  - "(1 <= h)" by "a0"
#  - "(10 >= h)" by "a1"

atoms = formula.get_atoms()
sorted_atoms = sorted(atoms)
bool_variables = set(Symbol("a" + str(i), BOOL) for (i, a) in enumerate(sorted_atoms))
form_2_abstr = dict(zip(bool_variables, atoms))
# formula           = (1 <= h) & (h <= 10)
# abstract_formula  = a0 & a1
abstract_formula = formula.substitute(form_2_abstr)

# Call to SETTA method

abstr_2_form = dict(zip(atoms, bool_variables))
fp = abstract_formula.substitute(abstr_2_form)
# abstract_formula  = a0 & a1
# fp           = (1 <= h) & (h <= 10)

print(formula)
print(fp)
print(abstract_formula)

((1 <= h) & (h <= 10))
(a0 & a1)
((1 <= h) & (h <= 10))


In [48]:
with Solver(logic="QF_LIA") as solver:
    solver.add_assertion(formula)
    if not solver.solve():
        print("Domain is not SAT!!!")
        exit()
    if solver.solve():
        #for l in letters:
        for l in int_variables: 
            print("%s = %s" %(l, solver.get_value(l)))
    else:
        print("No solution found")

h = 1


In [54]:
print(sorted_atoms[0])
print(Not(sorted_atoms[0]))

(h <= 10)
(! (h <= 10))


In [55]:
with Solver(logic="QF_LIA") as solver:
    solver.add_assertion(Not(sorted_atoms[0]))
    if not solver.solve():
        print("Domain is not SAT!!!")
        exit()
    if solver.solve():
        #for l in letters:
        for l in int_variables: 
            print("%s = %s" %(l, solver.get_value(l)))
    else:
        print("No solution found")

h = 11
